In [0]:
# %% [code]
####Import Library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense,ZeroPadding2D, Activation, BatchNormalization

from keras.callbacks import CSVLogger, ReduceLROnPlateau, EarlyStopping

import glob
import cv2
from keras import optimizers
from keras.callbacks import ModelCheckpoint



# %% [code]

###Define Constants
FAST_RUN = False
IMAGE_WIDTH=227
IMAGE_HEIGHT=227
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
batch_size = 32
epochs = 15

# %% [code]
pip install split-folders

# %% [code]
pip install split-folders tqdm

# %% [code]
import split_folders

split_folders.ratio('../input/plantvillage-dataset/color', output="output", seed=1337, ratio=(.7, .15,.15))



# %% [code]

train_data_directory  = "./output/train"
validation_data_directory  = "./output/val"
test_data_directory="./output/test"
num_of_classes_on_trainDataset=print(len(glob.glob(train_data_directory +"/*")))
num_of_classes_on_valDataset=print(len(glob.glob(validation_data_directory+"/*")))
num_of_classes_on_testDataset=print(len(glob.glob(test_data_directory+"/*")))




# %% [code]
##Image Preprocessing

train_datagenerator = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagenerator = ImageDataGenerator(rescale=1./255)



training_set = train_datagenerator.flow_from_directory(train_data_directory ,
                                                 target_size=IMAGE_SIZE,
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

valid_set = valid_datagenerator.flow_from_directory(validation_data_directory,
                                            target_size=IMAGE_SIZE,
                                            batch_size=batch_size,
                                            class_mode='categorical')

test_datagenerator = ImageDataGenerator(rescale=1./255)
test_set = test_datagenerator.flow_from_directory(
    test_data_directory, 
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

# %% [code]
number_of_classes= print(len(glob.glob(train_data_directory +"/*")))
number_of_train_images=training_set.samples
print(number_of_train_images)
number_of_val_images=valid_set.samples
print(number_of_val_images)
number_of_test_images=test_set.samples
print(number_of_test_images)






# %% [code]
print(training_set.class_indices)

# %% [code]
list_of_classes = list(training_set.class_indices.keys())
print(list_of_classes)


# %% [code]

##See Random sample image

sample_type= random.choice(list_of_classes)
print(sample_type)

sample_random_img= random.choice(os.listdir(train_data_directory+"/"+sample_type))
print(sample_random_img)

image=load_img(train_data_directory+"/"+sample_type+"/"+sample_random_img)
plt.imshow(image)


# %% [code]

##Build Model
model = Sequential()

# Layer 1
model.add(Conv2D(96, kernel_size=(11,11), strides= 4, padding= 'valid', activation= 'relu',
                        input_shape= (227,227,3), kernel_initializer= 'he_normal'))
model.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),padding= 'valid', data_format= None))
model.add(BatchNormalization())



# Layer 2
model.add(Conv2D(256, kernel_size=(5,5), strides= 1,  padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
model.add(MaxPooling2D(pool_size=(3,3), strides= (2,2), padding= 'valid', data_format= None)) 
model.add(BatchNormalization())



# Layer 3

model.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))


model.add(BatchNormalization())
# Dropout
model.add(Dropout(0.5))

#layer 4

model.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

model.add(BatchNormalization())
# Dropout
model.add(Dropout(0.5))

# Layer 5

model.add(Conv2D(256, kernel_size=(3,3), strides= 1,  padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

model.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),  padding= 'valid', data_format= None))

# Dropout
model.add(Dropout(0.5))
          
          
# Layer 6

model.add(Flatten())

model.add(Dense(4096, activation= 'relu'))
model.add(Dropout(0.5))
          # Batch Normalisation
model.add(BatchNormalization())

# Layer 7
model.add(Dense(4096, activation= 'relu'))
model.add(Dropout(0.5))
          
model.add(BatchNormalization())

# Layer 8
model.add(Dense(1000, activation= 'relu'))
model.add(Dropout(0.5))
          
# Batch Normalisation
model.add(BatchNormalization())

# final layer
model.add(Dense(38, activation= 'softmax'))


model.summary()


# %% [code]

#model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
 #                   loss='categorical_crossentropy',
  #                  metrics=['accuracy'])
#model.compile(optimizer=optimizers.SGD(lr=0.001, momentum=0.9, decay=0.005),
 
    #loss='categorical_crossentropy',
     #         metrics=['accuracy'])
import tensorflow as tf
from tensorflow.python.keras.optimizers import Adam  
    
opt = tf.keras.optimizers.Adam(lr = 1e-4, decay = 1e-6)
model.compile(loss="binary_crossentropy", optimizer = opt,metrics = ["accuracy"])


# %% [code]
 
##Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau ,ModelCheckpoint

#earlystop = EarlyStopping(min_delta=0.001, patience=10)
#learning_rate_reduction = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

#callbacks_list = [learning_rate_reduction,earlystop ]

#lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
#early_stopper = EarlyStopping(min_delta=0.001, patience=10)

checkpoint = ModelCheckpoint("AlexNet.h5", monitor = 'accuracy', verbose = 1, save_best_only = True, save_weights_only = False, mode = 'auto', period = 1)

history_train= model.fit_generator(training_set,
                         steps_per_epoch=number_of_train_images//batch_size,
                         validation_data=valid_set,
                         epochs=epochs,
                         validation_steps=number_of_val_images//batch_size,
                         callbacks=[checkpoint],
                          verbose=1
                         )

filepath="AlexNet_Model.hdf5"
model.save(filepath)


# %% [code]
import matplotlib.pyplot as plt
xmin = 0
xmax = 15
ymin = 0.0
ymax = 1.0
acc = history_train.history['accuracy']
val_acc = history_train.history['val_accuracy']
loss = history_train.history['loss']
val_loss = history_train.history['val_loss']

epochs = range(len(acc))

axes = plt.gca()
axes.set_xlim([xmin,xmax])
axes.set_ylim([ymin,ymax])

plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# %% [code]
print("[INFO] Calculating model accuracy")
scores = model.evaluate_generator(generator=valid_set
)
print(f"Test Accuracy: {scores[1]*100}")

# %% [code]
from keras.preprocessing import image
sample_type= random.choice(list_of_classes)
print(sample_type)

sample_random_img= random.choice(os.listdir(test_data_directory+"/"+sample_type))


new_image=load_img(test_data_directory+"/"+sample_type+"/"+sample_random_img ,target_size=(227, 227))


img = image.img_to_array(new_image)
img = np.expand_dims(img, axis=0)
img = img/255

print("Following is our prediction:")
prediction = model.predict(img)
d = prediction.flatten()
j = d.max()
for index,item in enumerate(d):
    if item == j:
        class_name = list_of_classes[index]
    

img_prob =model.predict_proba(img)
print(img_prob.max() )

plt.figure(figsize = (4,4))
plt.imshow(new_image)
plt.axis('off')
plt.title(class_name)
plt.show()